# Wikipics demo

In [1]:
from pathlib import Path
from typing import NamedTuple
from ipywidgets.widgets import Image, Layout, HBox

import httpx

from wikipics import get_sample_url, get_sample_urls

class ImageRecord(NamedTuple):
    pixels: bytes
    name: str
    size: int

In [2]:
def fetch(url) -> ImageRecord:
    resp = httpx.get(url)
    resp.raise_for_status()
    name = Path(url).name
    return ImageRecord(resp.content, name, len(resp.content))

In [3]:
#img_rec = fetch(get_sample_url(1_000_000))
#Image(value=img_rec.pixels)

In [4]:
%%time
img_widgets = []
qty = 10
scale = f'{100//qty}%'
urls = get_sample_urls(2_000_000, qty)
total_bytes = 0
for url in urls:
    img_rec = fetch(url)
    print(f'{img_rec.size:12_} bytes | {img_rec.name}')
    total_bytes += img_rec.size
    img_widgets.append(Image(value=img_rec.pixels, layout=Layout(width=scale, height=scale)))

print(f'TOTAL BYTES: {total_bytes:_}')
HBox(img_widgets)

   1_969_876 bytes | Yogapith%2C_Mayapur.jpg
   1_992_966 bytes | Aeshna_cyanea_freshly_slipped_with_time.jpg
   2_010_689 bytes | Morocco_Africa_Flickr_Rosino_December_2005_84514010.jpg
   2_024_436 bytes | Fritillaria_meleagris_LJ_barje2.jpg
   2_006_701 bytes | Dhow_znz.jpg
   2_015_977 bytes | Litoria_infrafrenata_-_Julatten.jpg
   1_974_272 bytes | Panorpa_alpina_2_Luc_Viatour.jpg
   2_020_431 bytes | Kaiseradler_Aquila_heliaca_2_amk.jpg
   2_028_394 bytes | Busterkeaton.jpg
   2_007_922 bytes | AporiaCrataegi.jpg
TOTAL BYTES: 20_051_664
CPU times: user 742 ms, sys: 278 ms, total: 1.02 s
Wall time: 16.3 s


In [5]:
class Gallery:
    def __init__(self, num_images):
        scale = f'{100//num_images}%'
        self.img_layout = Layout(width=scale, height=scale)
        no_img = open('no-image.png', 'rb').read()
        self.img_widgets = [Image(value=no_img, layout=self.img_layout)] * num_images
        self.box = HBox(self.img_widgets)
        self.size = 0

    def display(self):
        display(self.box)

    def update(self, index, pixels):
        self.size += len(pixels)
        self.img_widgets[index] = Image(value=pixels, alt=f'({index})', layout=self.img_layout)
        self.box.children = self.img_widgets
        print(f'({index+1:2}){img_rec.size:12_} bytes | {img_rec.name}')


In [6]:
%%time
num_images = 10
gallery = Gallery(num_images)
gallery.display()

urls = get_sample_urls(2_000_000, num_images)

for i, url in enumerate(urls):
    img_rec = fetch(url)
    total_bytes += img_rec.size
    gallery.update(i, img_rec.pixels)

print(f'TOTAL BYTES: {gallery.size:_}')

( 1)   2_017_212 bytes | Mantoux_tuberculin_skin_test.jpg
( 2)   2_004_607 bytes | Porphyrio_indicus_viridis_-_Bueng_Boraphet%2C_Thailand.jpg
( 3)   2_028_394 bytes | Busterkeaton.jpg
( 4)   1_992_966 bytes | Aeshna_cyanea_freshly_slipped_with_time.jpg
( 5)   1_972_894 bytes | Barack_Obama_family_portrait_2011.jpg
( 6)   2_024_436 bytes | Fritillaria_meleagris_LJ_barje2.jpg
( 7)   1_974_272 bytes | Panorpa_alpina_2_Luc_Viatour.jpg
( 8)   1_994_699 bytes | BennyTrapp_Hyla_intermedia_Italien.jpg
( 9)   2_005_363 bytes | Marmot-edit1.jpg
(10)   2_010_325 bytes | Rana_esculenta_on_Nymphaea_edit.JPG
TOTAL BYTES: 20_025_168
CPU times: user 781 ms, sys: 285 ms, total: 1.07 s
Wall time: 16.6 s


In [7]:
%%time
from concurrent import futures

num_images = 10
gallery = Gallery(num_images)
gallery.display()

urls = get_sample_urls(2_000_000, num_images)
with futures.ThreadPoolExecutor() as pool:
    img_records = pool.map(fetch, urls)
    for i, img_rec in enumerate(img_records):
        gallery.update(i, img_rec.pixels)

print(f'TOTAL BYTES: {gallery.size:_}')

( 1)   2_019_740 bytes | Bald.eagle.closeup.arp-sh.750pix.jpg
( 2)   2_020_431 bytes | Kaiseradler_Aquila_heliaca_2_amk.jpg
( 3)   1_977_558 bytes | Frederic_Edwin_Church_-_Aurora_Borealis_-_Google_Art_Project.jpg
( 4)   1_974_272 bytes | Panorpa_alpina_2_Luc_Viatour.jpg
( 5)   1_992_966 bytes | Aeshna_cyanea_freshly_slipped_with_time.jpg
( 6)   1_980_243 bytes | 17_Years_of_Sekar_Jepun_2014-11-01_48.jpg
( 7)   1_976_553 bytes | Photographing_a_model.jpg
( 8)   1_969_876 bytes | Yogapith%2C_Mayapur.jpg
( 9)   2_010_689 bytes | Morocco_Africa_Flickr_Rosino_December_2005_84514010.jpg
(10)   1_977_894 bytes | Greyhound_Racing_2_amk.jpg
TOTAL BYTES: 19_900_222
CPU times: user 11 s, sys: 277 ms, total: 11.2 s
Wall time: 2.85 s


In [8]:
%%time
from IPython.display import display
from concurrent import futures

num_images = 10
gallery = Gallery(num_images)
gallery.display()
urls = get_sample_urls(2_000_000, num_images)

with futures.ThreadPoolExecutor() as pool:
    tasks = [pool.submit(fetch, url) for url in urls]
    for i, future in enumerate(futures.as_completed(tasks)):
        img_rec = future.result()
        gallery.update(i, img_rec.pixels)
        
print(f'TOTAL BYTES: {gallery.size:_}')

( 1)   1_975_417 bytes | Dolceacqua43_-_Artista_locale_mentre_dipinge_un_acquarello.jpg
( 2)   2_020_431 bytes | Kaiseradler_Aquila_heliaca_2_amk.jpg
( 3)   2_004_178 bytes | Thiodina_puerpera_female_02.jpg
( 4)   2_006_701 bytes | Dhow_znz.jpg
( 5)   1_994_699 bytes | BennyTrapp_Hyla_intermedia_Italien.jpg
( 6)   2_004_828 bytes | USA_10096-7-8_HDR_Antelope_Canyon_Luca_Galuzzi_2007.jpg
( 7)   1_984_907 bytes | Glasshouse_and_fountain_at_lalbagh.jpg
( 8)   1_972_509 bytes | Papilio_demoleus_ALT_by_kadavoor.jpg
( 9)   2_004_607 bytes | Porphyrio_indicus_viridis_-_Bueng_Boraphet%2C_Thailand.jpg
(10)   2_018_402 bytes | Palais_Garnier%27s_grand_salon%2C_12_February_2008.jpg
TOTAL BYTES: 19_986_679
CPU times: user 7.91 s, sys: 266 ms, total: 8.18 s
Wall time: 2.49 s
